In [1]:
!pip install hubsante_model
!ls


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Conversion.ipynb   converter          pytest.ini         setup.py
README.md          converter.egg-info requirements.txt   tests


In [2]:
import hubsante_model
FILE = '../src/main/resources/sample/examples/RC-EDA/RC-EDA-FuiteDeGaz-AliceGregoireNormand.json'
# '../src/main/resources/sample/examples/RS-EDA/RS-EDA-SMUR_FuiteDeGaz_AliceGregoireNormand.01.json'

In [3]:
import json
from pprint import pprint

with open(FILE) as json_data:
    # Input message
    input_json = json.load(json_data)
    pprint(input_json)

    # Message's model
    model = next(iter(input_json))
    print(model)
    
    # Object loading -> find right class based on model key and load only Usecase class
    # ToDo: be able to have just a top EDXLMessage class to load all objects
    modelClass = model[0].upper() + model[1:]  # createCase key to CreateCase class
    usecaseMessage = getattr(hubsante_model, modelClass).from_dict(input_json[model])
    pprint(usecaseMessage)

{'createCase': {'caseId': 'fr.health.samu440.DRFR154402413800236',
                'creation': '2024-05-18T18:15:00+02:00',
                'initialAlert': {'attachment': [{'URI': 'https://hub.esante.gouv.fr/resourceExample'}],
                                 'callTaker': {'calltakerContact': {'channel': 'PERSONNE',
                                                                    'detail': '0671830587',
                                                                    'type': 'TEL'},
                                               'calltakerId': '1234',
                                               'controlRoom': 'CRRA44',
                                               'organization': 'fr.health.samu440',
                                               'role': 'ARM'},
                                 'caller': {'callbackContact': {'channel': 'PERSONNE',
                                                                'detail': '+33671830523',
                                       

In [4]:
# API: receive RC-EDA and converts to RS-EDA

In [5]:
# Method 1: using objects => OK but requires to rewrite ALL the fields and focuses on output being OK not keeping all input data
with open(FILE) as json_data:
    # Input RC-EDA create_case content
    inMessage = hubsante_model.CreateCaseWrapper.from_json(json_data.read()).create_case
    
    # Output RS-EDA create_case_health content
    outMessage = hubsante_model.CreateCaseHealth(
        caseId=inMessage.case_id,
        senderCaseId=inMessage.sender_case_id,
        creation=inMessage.creation,
        perimeter="AMU", # ok ?
        interventionType="T1", # ok ?
        qualification=hubsante_model.health.models.qualification.Qualification.from_dict(inMessage.qualification.to_dict()),
        location=hubsante_model.health.models.location.Location.from_dict(inMessage.location.to_dict()),
        initialAlert=hubsante_model.health.models.alert.Alert.from_dict(inMessage.initial_alert.to_dict()),
        owner="fr.health.samu950", # ok ?
        patient=None, # or [] or not present
        medicalNote=None, # or [] or not present
        decision=None, # or [] or not present
        additionalInformation=inMessage.additional_information
    )
    pprint(outMessage)

CreateCaseHealth(case_id='fr.health.samu440.DRFR154402413800236', sender_case_id='DRFR154402413800236', creation='2024-05-18T18:15:00+02:00', perimeter='AMU', intervention_type='T1', qualification=Qualification(origin=None, risk_threat=[RiskThreat(code='R13', label="Risque d'explosion, présence de gaz")], whats_happen=WhatsHappen(code='C09.03.00', label='Fuite de gaz'), location_kind=LocationKind(code='L01.01.01', label="Maison particulière, pavillon, à l'intérieur"), health_motive=HealthMotive(code='M03.10', label='Malaise avec perte de connaissance initiale'), details=None), location=Location(name=None, external_location_id=None, detailed_address=DetailedAddress(highway=None, complete='Rue des Marins', number='3', way_name=WayName(complete='Rue des Marins', type='Rue', name='Des Marins')), city=City(name='Bouaye', insee_code='44018'), access=None, geometry=Geometry(datetime='2024-05-18T18:17:00+02:00', point=Point(coord=Coord(lat=47.147005, lon=-1.690548, height=None, precision='ADRE

In [6]:
def delete_paths(target_dict, paths):
    """
    Safely deletes keys in a dictionary based on dot-separated paths.

    :param target_dict: The dictionary to modify.
    :param paths: A list of dot-separated paths (e.g., "a.b.c").
    """
    def delete_recursively(d, keys):  # keys is the list of keys along a given path
        if not keys or not isinstance(d, dict):
            return
        
        key = keys[0]
        if len(keys) == 1:
            # If we are at the last key, delete it
            if key not in d:
                print(f"{key} not found")
            else:
                d.pop(key)
        else:
            # Recur for the next level
            if key in d:
                delete_recursively(d[key], keys[1:])
                # Remove the key if the nested dict becomes empty
                if isinstance(d[key], dict) and not d[key]:
                    d.pop(key)
            else:
                print(f"{key} not found")
    
    for path in paths:
        keys = path.split(".")  # Convert dot-separated path to list of keys
        delete_recursively(target_dict, keys)

# Example Usage
data = {
    "a": {
        "b": {
            "c": 1,
            "d": 2
        },
        "e": 3
    },
    "f": 4
}

paths_to_delete = ["a.b.c", "a.e", "f", "g", "a.h"]

delete_paths(data, paths_to_delete)

print(data)  # Output: {'a': {'b': {'d': 2}}}


g not found
h not found
{'a': {'b': {'d': 2}}}


In [8]:
def format_object(obj, indent=0):
    """
    Recursively formats an object into a readable string by detecting its attributes.

    :param obj: The object to format.
    :param indent: The current indentation level (for recursion).
    :return: A formatted string.
    """
    indent_str = "  " * indent
    if hasattr(obj, '__dict__'):  # Check if the object has attributes
        attributes = vars(obj)
        if not attributes:
            return f"{indent_str}No attributes to display."
        
        formatted_lines = []
        
        if indent == 0:
            # Add first class name (others will be found from attribute of parent)
            formatted_lines.append(f"{indent_str}{obj.__class__.__name__.title()}:")
        
        for field, value in attributes.items():
            if value is None:  # Skip fields with None
                continue
            if hasattr(value, '__dict__'):  # If the value is an object, recurse
                formatted_lines.append(f"{indent_str}- {field.replace('_', ' ').title()}:")
                formatted_lines.append(format_object(value, indent=indent + 1))
            else:  # Handle primitive values
                formatted_lines.append(f"{indent_str}- {field.replace('_', ' ').title()}: {value}")
        return "\n".join(formatted_lines)
    else:
        # Handle non-object cases like lists, dictionaries, or primitives
        if isinstance(obj, list):
            return "\n".join(f"{indent_str}- {format_object(item, indent + 1)}" for item in obj)
        elif isinstance(obj, dict):
            formatted_lines = []
            for key, value in obj.items():
                formatted_lines.append(f"{indent_str}- {key.replace('_', ' ').title()}:")
                formatted_lines.append(format_object(value, indent + 1))
            return "\n".join(formatted_lines)
        else:
            return f"{indent_str}{obj}"  # Base case for primitives

# Example usage
class Victims:
    def __init__(self, count, main_victim, freetext):
        self.count = count
        self.main_victim = main_victim
        self.freetext = freetext

class Incident:
    def __init__(self, type, location, victims):
        self.type = type
        self.location = location
        self.victims = victims

victims = Victims(count='PLUSIEURS', main_victim='ENFANT', freetext=None)
incident = Incident(type='Accident', location='School', victims=victims)

print(format_object(incident))

Incident:
- Type: Accident
- Location: School
- Victims:
  - Count: PLUSIEURS
  - Main Victim: ENFANT


In [9]:
# Method 2: just converting JSON (and creating object at the end) as the variations should be very basic and we don't want to bother with controls / miss any data etc
with open(FILE) as json_data:
    # Input RC-EDA create_case content
    in_str = json_data.read()
    in_json = json.loads(in_str)['createCase']
    in_message = hubsante_model.CreateCaseWrapper.from_json(in_str).create_case

    # Build output JSON
    out_json = json.loads(in_str)['createCase']  # to have an independant copy
    # - Deletions
    delete_paths(out_json, [
        "qualification.victims",  # Pas le mettre ailleurs ?!
        "attachment",
        "referenceVersion",
        "freetext",  # Pas le mettre ailleurs ?!
        "location.geometry.point.coord.heading",
        "location.geometry.point.coord.speed",
        "location.geometry.sketch",
        "location.country",
        "location.ID",  # ID devrait être Id, ça va pas poser plus tard pour les messages opérationnels ensuite ? On enverra pas les liens ?
        "location.locLabel",
        "location.city.detail",  # Pas les mettre ailleurs ?
        "initialAlert.id",
        "initialAlert.reporting",  # Le mettre ailleurs ?
        "initialAlert.qualification",
        "initialAlert.location",
        "initialAlert.callTaker",  # Le mettre ailleurs ?
        "newAlert"
    ])
    # - Updates
    # ToDo: improve controls to make sure fields exist
    out_json['owner'] = 'fr.health.samu950'  # ToDo: retrieve from EDXL-DE envelope
    
    if 'victims' in in_json['qualification'] and 'initialAlert' in in_json:
        if not 'notes' in in_json['initialAlert']:
            in_json['initialAlert']['notes'] = []
        out_json['initialAlert']['notes'].append({"freetext": f"{out_json['initialAlert'].get('freetext', '')}\n{format_object(in_message.qualification.victims)}"})
    
    # ToDo: implement all rules
    
    # Output RS-EDA create_case_health content
    outMessage = hubsante_model.CreateCaseHealth.from_dict(out_json)
    pprint(out_json)
    pprint(outMessage)

attachment not found
freetext not found
heading not found
speed not found
sketch not found
ID not found
locLabel not found
detail not found
newAlert not found
{'caseId': 'fr.health.samu440.DRFR154402413800236',
 'creation': '2024-05-18T18:15:00+02:00',
 'initialAlert': {'attachment': [{'URI': 'https://hub.esante.gouv.fr/resourceExample'}],
                  'caller': {'callbackContact': {'channel': 'PERSONNE',
                                                 'detail': '+33671830523',
                                                 'type': 'TEL'},
                             'callerContact': {'channel': 'PERSONNE',
                                               'detail': '+33671830523',
                                               'type': 'TEL'},
                             'communication': 'AUCUNE',
                             'detailedName': {'complete': 'Marie Dubois',
                                              'firstName': 'Marie',
                                          

In [10]:
d1 = {'a': 3}
d2 = {'a': 5}
{**d1, **d2}

{'a': 5}